# Bert Centralised Learning

Set the module directory to import python files (RUN JUST ONCE)

In [1]:

import os
os.chdir('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your project
import sys
sys.path.append('/home/victor/_bcfl/fabric-federated-learning/federated-learning')  # Replace with the path to your models directory
print(sys.path)

%load_ext autoreload
%autoreload 2



['/home/victor/_bcfl/fabric-federated-learning/federated-learning/client/notebooks', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python311.zip', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/lib-dynload', '', '/home/victor/anaconda3/envs/bcfl-fabric/lib/python3.11/site-packages', '/home/victor/_bcfl/fabric-federated-learning/federated-learning']


In [2]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### Initialize variables

In [10]:
from client.model.bert_tiny import get_bert_tiny_tokenizer

MAX_LEN = 512
TRAIN_BATCH_SIZE = 4
EVAL_BATCH_SIZE = 2
TEST_BATCH_SIZE = 8
TRAIN_SIZE = 0.8
EVAL_SIZE = 0.2
NUM_EPOCHS = 10
LEARNING_RATE = 1e-05
SEED = 200
tokenizer = get_bert_tiny_tokenizer()
twitter_dep_train = 'client/data/datasets/twitter_dep/twitter_dep_train.csv'
twitter_dep_test = 'client/data/datasets/twitter_dep/twitter_dep_test.csv'



### Loading Reddit Depression Dataloaders

In [11]:
from client.data.twitter_dep import get_twitter_dep_dataloaders


train_loader, eval_loader = get_twitter_dep_dataloaders(twitter_dep_train, tokenizer, 
                                                                    train_size=TRAIN_SIZE,
                                                                    eval_size=EVAL_SIZE,
                                                                    train_batch_size=TRAIN_BATCH_SIZE, 
                                                                    eval_batch_size=EVAL_BATCH_SIZE, 
                                                                    max_len=MAX_LEN, 
                                                                    seed=SEED)


### Creating Bert Tiny Model

In [12]:
from client.model.bert_tiny import get_bert_tiny_model

bert_tiny = get_bert_tiny_model(device=device)
print(bert_tiny)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 128, padding_idx=0)
      (position_embeddings): Embedding(512, 128)
      (token_type_embeddings): Embedding(2, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, e

### Loading Transformers Trainer

In [13]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
from client.data.twitter_dep import get_twitter_dep_datasets, get_twitter_dep_test_dataset

train_dataset, eval_dataset = get_twitter_dep_datasets(twitter_dep_train, tokenizer, TRAIN_SIZE, EVAL_SIZE, MAX_LEN, SEED)
test_dataset = get_twitter_dep_test_dataset(twitter_dep_test, tokenizer, MAX_LEN, SEED)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    metric = evaluate.load("accuracy")
    return metric.compute(predictions=predictions, references=labels)

training_args = TrainingArguments(output_dir="client/models/bert-tiny/", evaluation_strategy="epoch")


trainer = Trainer(
    model=bert_tiny,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

### Train model with Trainer 

In [14]:
trainer.train()

  0%|          | 0/744 [00:00<?, ?it/s]

  0%|          | 0/62 [00:00<?, ?it/s]

{'eval_loss': 0.4992581009864807, 'eval_accuracy': 0.7782258064516129, 'eval_runtime': 2.9603, 'eval_samples_per_second': 167.552, 'eval_steps_per_second': 20.944, 'epoch': 1.0}


  0%|          | 0/62 [00:00<?, ?it/s]

{'eval_loss': 0.47255343198776245, 'eval_accuracy': 0.7943548387096774, 'eval_runtime': 2.7936, 'eval_samples_per_second': 177.546, 'eval_steps_per_second': 22.193, 'epoch': 2.0}
{'loss': 0.5215, 'learning_rate': 1.639784946236559e-05, 'epoch': 2.02}


  0%|          | 0/62 [00:00<?, ?it/s]

{'eval_loss': 0.4657380282878876, 'eval_accuracy': 0.7943548387096774, 'eval_runtime': 3.0177, 'eval_samples_per_second': 164.365, 'eval_steps_per_second': 20.546, 'epoch': 3.0}
{'train_runtime': 66.3665, 'train_samples_per_second': 89.593, 'train_steps_per_second': 11.21, 'train_loss': 0.4979045109082294, 'epoch': 3.0}


TrainOutput(global_step=744, training_loss=0.4979045109082294, metrics={'train_runtime': 66.3665, 'train_samples_per_second': 89.593, 'train_steps_per_second': 11.21, 'train_loss': 0.4979045109082294, 'epoch': 3.0})

### Test the trained model

In [2]:
from client.test import test_text_class

# Test the model
test_text_class(bert_tiny, test_loader, device=device)

ModuleNotFoundError: No module named 'client'